In [43]:
from google.genai import types # type: ignore
from google import genai
import pandas as pd
import sys
import os
notebook_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)
from app.GeminiClient import GeminiClient
import time
from app.MariaDBClient import MariaDBClient
from app.MarketScraper import MarketScraper
import pandas as pd
import asyncio

In [44]:
# https://www.listendata.com/2024/05/how-to-use-gemini-in-python.html
# Create API KEY: https://aistudio.google.com/app/u/0/apikey?pli=1

In [45]:
mariadbclient = MariaDBClient()
supermercados = mariadbclient.get_active_tables()
market_scraper = MarketScraper()
total_items = []
total_stats = []
for supermercado in supermercados:
    data = market_scraper.fetch_data(supermercado)
    items = market_scraper.get_items(data)
    print(supermercado, " : ", len(items))
    total_items.append(items)
    stas = market_scraper.get_stats(data)
    total_stats.append(stas)

total_items_df = pd.concat(total_items, ignore_index=True)
total_stats_df = pd.concat(total_stats, ignore_index=True)


atomo  :  367
blowmax  :  270
modomarket  :  406
segal  :  166


In [62]:
print(stas['elapsed_time_seconds'].iloc[0])


15.007821


In [46]:
new_items_df = total_items_df

In [47]:
old_items_df = mariadbclient.get_products()

In [48]:
new_only_df = new_items_df[~new_items_df['id_producto'].isin(old_items_df['id_producto'])] # type: ignore
common_df = new_items_df[new_items_df['id_producto'].isin(old_items_df['id_producto'])]
old_only_df = old_items_df[~old_items_df['id_producto'].isin(new_items_df['id_producto'])]
print(f"Nuevos productos: {new_only_df.shape[0]}")
print(f"Productos ya existentes: {common_df.shape[0]}")
print(f"Productos ya no existentes: {old_only_df.shape[0]}")

Nuevos productos: 0
Productos ya existentes: 1209
Productos ya no existentes: 0


In [49]:
old_only_df

,__ingestion_timestamp,nombre_crudo,id_producto,categoria,nombre,marca,descripcion,volumen,precio,id_supermercado,id_grupo,id_subgrupo,url


In [50]:
mariadbclient.delete_products(old_only_df)
print("Se eliminaron ",len(old_only_df), " productos")
mariadbclient.update_products(common_df)
print("Se actualizaron ",len(common_df), " productos")

Se eliminaron  0  productos
Se actualizaron  1209  productos


In [51]:
old_productos_to_update_subgroups_df = mariadbclient.get_products_by_groups(new_only_df)
new_productos_to_update_subgroups_df = new_only_df[['id_producto', 'categoria', 'marca', 'descripcion', 'volumen', 'id_supermercado']]
new_productos_to_update_subgroups_df = new_productos_to_update_subgroups_df.rename(columns={'id_supermercado': 'supermercado'})
new_productos_to_update_subgroups_df['supermercado'] = new_productos_to_update_subgroups_df['supermercado'].map({
    1:"Atomo",
    2:"Blowmax",
    3:"ModoMarket",
    4:"Segal",
    5:"Supera"
})
all_productos_to_upsert_subgroups_df = pd.concat([new_productos_to_update_subgroups_df, old_productos_to_update_subgroups_df], ignore_index=True)
all_productos_to_upsert_subgroups_df['producto_id'] = all_productos_to_upsert_subgroups_df['id_producto'].astype(str) + " - " + all_productos_to_upsert_subgroups_df['categoria'] + " - " + all_productos_to_upsert_subgroups_df['marca'] + " - " + all_productos_to_upsert_subgroups_df['descripcion'] + " - " + all_productos_to_upsert_subgroups_df['volumen'] + " - " + all_productos_to_upsert_subgroups_df['supermercado']


In [52]:
print("Hay ", len(old_productos_to_update_subgroups_df), " productos que comparten subgrupo con los productos nuevos")
print("Hay ", len(new_productos_to_update_subgroups_df), " productos nuevos que hay que asignar subgrupo")
print("Hay ", len(all_productos_to_upsert_subgroups_df), " productos que hay que actualizar su subgrupo")

Hay  0  productos que comparten subgrupo con los productos nuevos
Hay  0  productos nuevos que hay que asignar subgrupo
Hay  0  productos que hay que actualizar su subgrupo


In [53]:
grouped = all_productos_to_upsert_subgroups_df.groupby(['categoria', 'marca', 'volumen'])
listas_productos = [group['producto_id'].tolist() for _, group in grouped]

geminiclient = GeminiClient()
geminiclient.api_keys

total_elementos = sum(len(sublista) for sublista in listas_productos)
all_responses = []

indice = 0
for lista_productos in listas_productos:
    print("clasificando: ", str(lista_productos))
    
    with open('agrupacion.txt', 'a') as file: #!

        prompt = listas_productos[indice]
        try:
            if len(prompt) > 1:
                respuesta = geminiclient.ask(prompt)

                # for lista in respuesta:
                #     file.write(str(lista) + '\n') #!
                # file.write('----'*10 + '\n') #!

                time.sleep(4.1)
            else:
                respuesta = [prompt]
            all_responses.append(respuesta)
        except Exception as e:
            print(f"Error al procesar el prompt: {prompt}")
            print(f"Error: {e}")
    indice += 1

16m 39s

In [54]:
print("Gemini tiene que agrupar ",total_elementos, " productos")
total_agrupados = sum(len(subsublist) for sublist in all_responses for subsublist in sublist)
print("Gemini agrupo ",total_agrupados, " productos")


Gemini tiene que agrupar  0  productos
Gemini agrupo  0  productos


In [14]:
all_responses

[[['3090295548 - Arroces - ALA - PARBOIL - 1000 - Atomo'],
  ['996059818 - Arroces - ALA - LARGO FINO - 1000 - Atomo'],
  ['2772439016 - Arroces - ALA - INTEGRAL - 1000 - Atomo']],
 [['727038735 - Arroces - ALA - MOLINO CARNAROLI ESTUCHE - 500 - Atomo']],
 [['1514123841 - Arroces - APOSTOLES - LARGO FINO 00000 - 1000 - Atomo']],
 [['1398147232 - Arroces - BARBARA - LARGO FINO - 1000 - Blowmax']],
 [['2351976883 - Arroces - BARBARA - LARGO FINO - 500 - Blowmax']],
 [['189310520 - Arroces - DON MARCOS - LARGO FINO - 1000 - Segal'],
  ['1778123096 - Arroces - DON MARCOS - PARBOIL - 1000 - Segal']],
 [['2973007399 - Arroces - DOS HERMANOS - LARGO FINO - 1000 - ModoMarket'],
  ['3916362563 - Arroces - DOS HERMANOS - PARBOIL - 1000 - ModoMarket']],
 [['2194075035 - Arroces - EL GRANDE -  - 1000 - Blowmax']],
 [['1697171246 - Arroces - EL GRANDE -  - 500 - Blowmax']],
 [['267056816 - Arroces - GALLO - LARGO FINO SELEC - 1000 - Atomo',
   '1314536334 - Arroces - GALLO - LARGO FINO LARGO FIN - 

In [15]:
#TODO
#TODO EN el main si len(all_response) return, no hace falta seguir para abajo

In [16]:
olds_ids = set(old_productos_to_update_subgroups_df['id_producto'])
news_ids = set(new_items_df['id_producto'])

# Preparamos las estructuras de resultados
updates = []
descartados = []

# Creamos un diccionario para mapear rápidamente los IDs a sus índices en new_items_df
new_items_index = {id_producto: idx for idx, id_producto in enumerate(new_items_df['id_producto'])}

# Procesamos todas las respuestas
for respuesta in all_responses:
    for j, subgrupo in enumerate(respuesta):
        ids_productos = [item.partition(' - ')[0].strip() for item in subgrupo]  # .strip() para eliminar espacios
        
        for id_producto_str in ids_productos:
                id_producto = int(id_producto_str)
                
                if id_producto in olds_ids:
                    updates.append({ 
                        "id_producto": id_producto,
                        "id_subgrupo": j
                    })
                elif id_producto in news_ids:
                    idx = new_items_index[id_producto]
                    new_only_df.at[idx, 'id_subgrupo'] = j
                else:
                    descartados.append(id_producto_str)


In [17]:
new_only_df

,__ingestion_timestamp,nombre_crudo,id_producto,categoria,nombre,marca,descripcion,volumen,precio,id_supermercado,id_grupo,id_subgrupo,url
0,2025-05-01 18:10:41.937904,SHAMPOO HEAD & SHOULDER ANTI COMEZON 375 ML,3367199917,Shampoos,HEAD & SHOULDERS ANTI COMEZON 375,HEAD & SHOULDERS,ANTI COMEZON,375,7875.0,1,1714631061,0.0,https://atomoconviene.com/atomo-ecommerce/sham...
1,2025-05-01 18:10:41.937904,SHAMPOO FLIAR SUAVE BOMBA DELICIOSA 930 ML.,2651681944,Shampoos,SUAVE BOMBA DELICIOSA 930,SUAVE,BOMBA DELICIOSA,930,3265.0,1,1608173070,0.0,https://atomoconviene.com/atomo-ecommerce/sham...
2,2025-05-01 18:10:41.937904,SHAMPOO HEAD & SHOULDER ANTI COMEZON 180 ML,1439929977,Shampoos,HEAD & SHOULDERS ANTI COMEZON 180,HEAD & SHOULDERS,ANTI COMEZON,180,4703.0,1,4001080107,0.0,https://atomoconviene.com/atomo-ecommerce/sham...
3,2025-05-01 18:10:41.937904,SHAMPOO PANTENE ANTI CAIDA 510 ML.,4265273449,Shampoos,PANTENE ANTI CAIDA 510,PANTENE,ANTI CAIDA,510,8711.0,1,1065200442,0.0,https://atomoconviene.com/atomo-ecommerce/sham...
4,2025-05-01 18:10:41.937904,SHAMPOO PANTENE ANTI CAIDA 300 ML.,3853368190,Shampoos,PANTENE ANTI CAIDA 300,PANTENE,ANTI CAIDA,300,5372.0,1,2668965212,0.0,https://atomoconviene.com/atomo-ecommerce/sham...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207,2025-05-01 18:12:55.182086,TRESEMMÉ SHAMPOO 400ml BAJO POO NUTRICIÓN,530611243,Shampoos,TRESEMME BAJO POO NUTRICION 400,TRESEMME,BAJO POO NUTRICION,400,4569.0,4,2472537432,0.0,https://www.casa-segal.com/producto/tresemme-s...
1208,2025-05-01 18:12:55.182086,TRESEMMÉ SHAMPOO 400ml DETOX CAPILAR,3670746432,Shampoos,TRESEMME DETOX CAPILAR 400,TRESEMME,DETOX CAPILAR,400,4569.0,4,2472537432,1.0,https://www.casa-segal.com/producto/tresemme-s...
1209,2025-05-01 18:12:55.182086,TRESEMMÉ SHAMPOO 400ml HIDRATACIÓN PROFUNDA,4184477020,Shampoos,TRESEMME HIDRATACION PROFUNDA 400,TRESEMME,HIDRATACION PROFUNDA,400,4569.0,4,2472537432,2.0,https://www.casa-segal.com/producto/tresemme-s...
1210,2025-05-01 18:12:55.182086,TRESEMMÉ SHAMPOO 400ml REGENERACIÓN TRESPLEX,3660437582,Shampoos,TRESEMME REGENERACION TRESPLEX 400,TRESEMME,REGENERACION TRESPLEX,400,4569.0,4,2472537432,3.0,https://www.casa-segal.com/producto/tresemme-s...


In [18]:
updates

[]

In [19]:
descartados

['41730462']

In [20]:
mariadbclient.insert_into_productos(new_only_df)
mariadbclient.update_product_subgroups(updates)